In [ ]:
# pylint: disable=unused-import
import sys
from datetime import date
print(f"revision date: {date.today()} with Python {sys.version}")

revision date: 2025-01-29 with Python 3.8.20 (default, Sep  7 2024, 18:35:08) 
[GCC 11.4.0]


In [2]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

# THIS USES GLOBAL VARIABLES FOR KEY FILE
# AND FOR EXAMPLE ONLY OF PEM GENERATION

def make_private_pem(keyname="./private_key.pem"):
    # Save the keys to files (optional)
    with open(keyname, "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption(),
        ))

def make_public_pem(keyname="./public_key.pem"):
    with open(keyname, "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo,
        ))

def encrypt_message(message = b"YourSecretPassword"):
    # Encrypt a message with the public key
    ciphertext = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        )
    )
    return ciphertext

def decrypt_message(ciphertext):
    # Decrypt the message with the private key
    decrypted_message = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        )
    )
    return decrypted_message

# Generate RSA private and public keys
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# def make_public_key():
public_key = private_key.public_key()


In [ ]:

cyphertext = encrypt_message(b"this is it")
print(f"Encrypted message: {cyphertext}")
print(f"Decrypted message: {decrypt_message(cyphertext).decode()}")

make_private_pem()
make_public_pem()



Encrypted message: b"\xb1\xcf\xef\x87}\x83}s>\xd5z&\xe2\x9b\x9d\xaf\xf5AD\x98\x89\xfe\x80\xf2\xfe\x93\xf5{-\x7fL \x82)\xe2b\xd1I/\xdc2\xb1\xea1&q\xcd\xdf\xa3\xc3\xcd\x0b\xf1\x1e'\x14\xe4\x13Y\xb2\xe3x\x1brv\x93y\xeez\xee\x12Gp\xab\xe7/\x14\x15\xb8#G]\xdc\xac\x15\xc7\x95\xec{\xeaU8\xa9\x81@\xeb\xa5\xa3\x92\xd6ig\xe3\xd8\x1b\xf6\xb1v\xc0e\xad7\xd3C\x17g\x0c\xa1\xba\x98\xbe\x00\xd3r\xe7r\xa7\x06\xb0\xe7\x14\xc8$5\x11e \x1e>\xca\x0c(\xef\x83U\x15|\x1b\x06\xb1\x19\xdb\xc7\xfd\x97\xaf\xb3\xe1\x14k^\xfaZA\x81\x9cv[\x1c\x88\xe8\x8b\xa7\xff\x91\xbe\xdfP\x8a?Y\xe4\xc7\x1e2\x9b\x8fR\xd2A\xa8\x98\x90=\xed I\xaa\xae\xff\xe0{)=\xbc\xda\x17\x923:2q\xce\xc0\x9e'u\xd5\x88\x06\x0bx\xe9\xbb:\x19\x16\x8f'\x15\x9fV\xd14K#'+\xd5a\xe8\x95\xdc\xbfC0\xdek,m\x86\x91M\xc5\xb4\xac"
Decrypted message: this is it


In [4]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

# THIS DEOS A BETTER JOB AT ENCAPSULATING SECRET DATA FOR MEMORY LEAKS

def make_private_pem(keyname="private_key.pem"):
  # Generate RSA private and public keys
  private_key = rsa.generate_private_key(
      public_exponent=65537,
      key_size=2048,
  )

  # Save the keys to files (optional)
  with open(keyname, "wb") as f:
      f.write(private_key.private_bytes(
          encoding=serialization.Encoding.PEM,
          format=serialization.PrivateFormat.PKCS8,
          encryption_algorithm=serialization.NoEncryption(),
      ))
  return keyname

def make_public_pem(private_key_path, public_key_path="public_key.pem"):
  with open(private_key_path, "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,  # If the key is encrypted, provide the password here
        backend=default_backend()
    )
    public_key = private_key.public_key()
    with open(public_key_path, "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo,
        ))
  return public_key_path

def encrypt_message(public_key_path="./public_key.pem", ciphertext_file_path="ciphertext.bin", message=b"YourSecretPassword"):
    # Load the public key from a PEM file
    with open(public_key_path, "rb") as f:
        public_key = serialization.load_pem_public_key(f.read())

    # Encrypt a message with the public key
    ciphertext = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        )
    )
    # Save the ciphertext to a file (optional)
    with open(ciphertext_file_path, "wb") as f:
        f.write(ciphertext)
    print("Encrypted message:", ciphertext)

def decrypt_message(private_key_path, ciphertext_file_path):
  with open(private_key_path, "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,  # If the key is encrypted, provide the password here
        backend=default_backend()
    )

    # Read the ciphertext from a file
    with open(ciphertext_file_path, "rb") as f:
        ciphertext = f.read()

  plaintext = private_key.decrypt(
      ciphertext,
      padding.OAEP(
          mgf=padding.MGF1(algorithm=hashes.SHA256()),
          algorithm=hashes.SHA256(),
          label=None
      )
  )

  return plaintext

private_key_path = "./end-to-end_private_key.pem"
public_key_path = "./end-to-end_public_key.pem"
ciphertext = "end-to-end.bin"
private = make_private_pem(keyname=private_key_path)
public = make_public_pem(private_key_path, public_key_path)
print(f"Created {private} and {public}")

byte_text = b"absolutely!"  # Replace with your actual ciphertext
encrypt_message(public_key_path=public_key_path, ciphertext_file_path=ciphertext, message=byte_text)
decrypted_message = decrypt_message(private_key_path, ciphertext_file_path=ciphertext)
print(decrypted_message.decode('utf-8'))  # Decode the bytes to a string



Created ./end-to-end_private_key.pem and ./end-to-end_public_key.pem
Encrypted message: b"+.\x1eOeA\xd2\x04J\x0b\xb2\xb2\x179\xe3\xe3'\xab2H\xdf\xe0#\x1cq\x7f|\x1d\x93\xe0\xd8/ED\xbc\x07\xf6\xda\xfca\x93\xf3\xdf\xb8\xa4\xaa\xfc_\x0f\xd4=\xf7Bv)\x0e\xa3\xbe\xf9}\x06\xf8[O\x92_\xaf59Y\xc88\xb8\x9f\xc6\x00\x97,\xe8C\xcb\n<7\xbc<}\x86\xcd\xcb1H\xfd$V\xae\x9d]\x04\xf4+\x84\x8b\x91\x84`\xb6`\xb9\x9b@\xe7\x1b\xc8O6\xe9\x8e\xf2\x90bQ\xefh\x9a\x13{\xd1\x95\xab\xdch\xcb04n\x13\xcbi\x94d\x9a\x14<Tg\xbf.\xff\x81s\xaf\xfa5T\xe0\x9a+\x7f\xc9\x9a\xe1\xd0\xcc\xd5M\x8fs\xb3\r$\x1b\x84t\xbc\xc2M\x086&\xea\xdf\xe6\x11\x0cv\x1e(\n\x93\x94\xd0\xdc3K`D\x118\x83\xb5\x18\x01\xcb \x9d\xcd\xbb\xf1\x1e \x08\xf6\xe6\x16\xc6\xf8\xf6E\xbd\xa3h\x19N9\xbb_\x03\xc9\xfb e\xf2\x0b\xeb1I\x97\xc1\xfa\x91=\x90\x0c\xee\xbf\x05\x05\xd0\xfa\x08>A>\xb8f"
absolutely!
